In [4]:
import pandas as pd

df = pd.read_csv('cleaned_airbnb.csv')


In [8]:
import json
import pickle

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
# Print the shape and number of missing data per column.
print(f'This dataframe has {df.shape[0]} entries and {df.shape[1]} columns.')
print('----------')
print('Total missing entries per column:\n')
print(df.isna().sum())

# Print the head of the dataframe.
print('----------')
df.head()

This dataframe has 133585 entries and 12 columns.
----------
Total missing entries per column:

space                   0
description             0
city                    0
state                   0
room_type               0
price                   0
security_deposit        0
guests_included         0
extra_people            0
minimum_nights          0
maximum_nights          0
review_scores_rating    0
dtype: int64
----------


,space,description,city,state,room_type,price,security_deposit,guests_included,extra_people,minimum_nights,maximum_nights,review_scores_rating
0,"I like to call my place ""the penthouse"". It's ...",I like to think of my place as a cozy escape. ...,BROOKLYN,NY,Entire home/apt,87.0,0.0,1,0.0,4,1125,100.0
1,This room is one of two bedrooms open to book...,"Gorgeous, spacious, clean & cozy bedroom in th...",NEW YORK,NY,Private room,100.0,0.0,1,25.0,2,25,96.0
2,Our craftsman style home was aesthetically rem...,This cozy two bedroom apt is well lit. Then ap...,SEATTLE,WA,Entire home/apt,175.0,250.0,2,10.0,3,1125,100.0
3,There are two bathrooms on this floor where th...,The subways that is near with the house is the...,BROOKLYN,NY,Private room,80.0,0.0,2,0.0,30,1125,100.0
4,- Just renovated 3.5 bedroom / 2 bathroom pent...,Just updated 3.5 BR | 2 BA condo that features...,BOSTON,MA,Entire home/apt,600.0,650.0,5,40.0,2,270,99.0


In [11]:
df.drop(['space', 'description', 'room_type'], axis=1)

,city,state,price,security_deposit,guests_included,extra_people,minimum_nights,maximum_nights,review_scores_rating
0,BROOKLYN,NY,87.0,0.0,1,0.0,4,1125,100.0
1,NEW YORK,NY,100.0,0.0,1,25.0,2,25,96.0
2,SEATTLE,WA,175.0,250.0,2,10.0,3,1125,100.0
3,BROOKLYN,NY,80.0,0.0,2,0.0,30,1125,100.0
4,BOSTON,MA,600.0,650.0,5,40.0,2,270,99.0
...,...,...,...,...,...,...,...,...,...
133580,AUSTIN,TX,80.0,0.0,1,0.0,2,7,100.0
133581,COLUMBUS,OH,40.0,1500.0,1,5.0,1,1124,97.0
133582,NEW ORLEANS,LA,110.0,0.0,4,0.0,1,1125,97.0
133583,HALEIWA,HI,125.0,100.0,2,300.0,30,1125,96.0


In [14]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3, random_state=42)

train.shape, test.shape

((93509, 12), (40076, 12))

In [18]:
train, val = train_test_split(train, train_size=0.80, test_size=0.20,
                               random_state=42)

In [19]:
train.shape, test.shape, val.shape

((74807, 12), (40076, 12), (18702, 12))

In [25]:
features = ['security_deposit', 'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights', 'city', 'state']
target = 'price'

In [26]:

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [31]:
from lightgbm import LGBMClassifier
import category_encoders as ce
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    ce.OneHotEncoder(),
    SimpleImputer(),
    StandardScaler(), 
    DecisionTreeRegressor()
    
)

# Fit on train, score on val, predict on test
pipeline.fit(X_train, y_train)
print('Validation Accuracy', pipeline.score(X_val, y_val))
y_pred = pipeline.predict(X_test)


Validation Accuracy 0.18376496419175337


In [32]:
from sklearn.externals import joblib
joblib.dump(pipeline, 'placeholder.pkl')

['placeholder.pkl']